# Sammensetning av alt

#### Imports

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plts
from sklearn.metrics import accuracy_score, precision_score, recall_score
from LSTM import long_short_term_memory as LSTM
from rf import rf

#### Colors

In [2]:
rsrp_colors = ['#384959', '#6A89A7', '#88BDF2', '#BDDDFC']
rsrq_colors = ['#614419', '#B37E2E', '#DB9A39', '#FFB343']
sinr_colors = ['#135E4B', '#4CB572', '#A1D8B5', '#CCDCDB']

In [3]:
'''
    Reading the dataset and returning two pandas DataFrames:
        The first DataFrame containing every datapoint belonging to the given campaign (if any)
        The second DataFrame containing only one column - the mode (4G or 5G)
'''
def read_dataset(filename, campaign=None, features=None):
    df = pd.read_csv(filename)
    df = df.loc[:, ~df.columns.str.match('Unnamed')]
    df = df.loc[:, ~df.columns.str.match('Timestamp')]
    df = df.replace('DC', 1)
    df = df.replace('LTE', 0)
    df = df[df['campaign'].str.contains('Driving')]

    if campaign != None:
        df = df[df['campaign'] == campaign]

    if features == None:
        features = ['RSRP', 'SINR', 'RSRQ', 'SSS_RSRP', 'SSS_SINR', 'SSS_RSRQ', 'campaign']
    
    df, mode = df[features], df['Mode']
    return df, mode

## LSTM-prediction of signal strength indicators

In [4]:
# campaign='Interactivity_gaming_4G5G_TIM_Driving_Palasport_to_Via_Appia_End_No_Flight_Mode'

# rsrp_results = {}
# feature_selection = [['RSRP'], 
#                      ['RSRP', 'RSRQ'], ['RSRP', 'SINR'], ['RSRP', 'SINR', 'RSRQ'],
#                      ['RSRP', 'SSS_RSRP'], 
#                      ['RSRP', 'SINR', 'RSRQ', 'SSS_RSRP', 'SSS_SINR', 'SSS_RSRQ']]

# for features in feature_selection:
#     used_features = ', '.join(features)
#     feats = features.copy()
#     features.append('campaign')
#     X, mode = read_dataset('datasets/Op1_merged.csv', campaign=campaign, features=features)
#     rsrp_results[used_features] = LSTM(X=X, target_feature='RSRP', features=feats, n_steps_in=5, n_steps_out=1)

# rsrq_results = {}
# feature_selection = [['RSRQ'], 
#                      ['RSRQ', 'SINR'], ['RSRQ', 'RSRP'], ['RSRQ', 'RSRP', 'SINR'],
#                      ['RSRQ', 'SSS_RSRQ'], 
#                      ['RSRQ', 'RSRP', 'SINR', 'SSS_RSRP', 'SSS_SINR', 'SSS_RSRQ']]

# for features in feature_selection:
#     used_features = ', '.join(features)
#     feats = features.copy()
#     features.append('campaign')
#     X, mode = read_dataset('datasets/Op1_merged.csv', campaign=campaign, features=features)
#     rsrq_results[used_features] = LSTM(X=X, target_feature='RSRQ', features=feats, n_steps_in=5, n_steps_out=1)

# sinr_results = {}
# feature_selection = [['SINR'], 
#                      ['SINR', 'RSRQ'], ['SINR', 'RSRP'], ['SINR', 'RSRP', 'RSRQ'],
#                      ['SINR', 'SSS_SINR'], 
#                      ['SINR', 'RSRP', 'RSRQ', 'SSS_RSRP', 'SSS_SINR', 'SSS_RSRQ']]

# for features in feature_selection:
#     used_features = ', '.join(features)
#     feats = features.copy()
#     features.append('campaign')
#     X, mode = read_dataset('datasets/Op1_merged.csv', campaign=campaign, features=features)
#     sinr_results[used_features] = LSTM(X=X, target_feature='SINR', features=feats, n_steps_in=5, n_steps_out=2)

In [5]:
# df_rsrp_results = pd.DataFrame.from_dict(rsrp_results, orient='index')

# df_rsrp_results.reset_index(inplace=True)
# df_rsrp_results.rename(columns={'index': 'Combination'}, inplace=True)
# df_rsrp_results = df_rsrp_results.drop(['y_pred', 'y_true', 'loss', 'val_loss', 'LSTM', 'scaler_pred'], axis=1)
# df_rsrp_results

In [6]:
# df_rsrq_results = pd.DataFrame.from_dict(rsrq_results, orient='index')

# df_rsrq_results.reset_index(inplace=True)
# df_rsrq_results.rename(columns={'index': 'Combination'}, inplace=True)
# df_rsrq_results = df_rsrq_results.drop(['y_pred', 'y_true', 'loss', 'val_loss', 'LSTM', 'scaler_pred'], axis=1)
# df_rsrq_results

In [7]:
# df_sinr_results = pd.DataFrame.from_dict(sinr_results, orient='index')

# df_sinr_results.reset_index(inplace=True)
# df_sinr_results.rename(columns={'index': 'Combination'}, inplace=True)
# df_sinr_results = df_sinr_results.drop(['y_pred', 'y_true', 'loss', 'val_loss', 'LSTM', 'scaler_pred'], axis=1)
# df_sinr_results

In [8]:
campaign='Interactivity_gaming_4G5G_TIM_Driving_Palasport_to_Via_Appia_End_No_Flight_Mode'

df, mode = read_dataset('datasets/Op1_merged.csv', campaign=campaign)
test_idx = 3 * df.shape[0] // 4

train, mode_train, test, mode_test = df[0 : test_idx], mode[0 : test_idx], df[test_idx ::], mode[test_idx ::]

results = {}
target_features = ['RSRP', 'SINR', 'RSRQ']
feature_selection = [['RSRP'], ['SINR'], ['RSRQ'], 
                     ['RSRP', 'SINR'], ['RSRP', 'RSRQ'], ['SINR', 'RSRQ'], ['RSRP', 'SINR', 'RSRQ'],
                     ['RSRP', 'SSS_RSRP'], ['SINR', 'SSS_SINR'], ['RSRQ', 'SSS_RSRQ'], 
                     ['RSRP', 'SINR', 'RSRQ', 'SSS_RSRP', 'SSS_SINR', 'SSS_RSRQ']]

In [9]:
for target in target_features:
    for features in feature_selection:
        if features.__contains__(target):
            used_features = ', '.join(features)
            used_features = used_features + ' -> ' + target
            print(used_features)
            features.append('campaign')
            results[used_features] = LSTM(X=df[features], test_idx=test_idx, target_feature=target, features=features, n_steps_in=5, n_steps_out=1)

RSRP -> RSRP


/Users/ihneskarbovik/Documents/master/master-greier/master/LSTM.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[features] = scaler.fit_transform(X[features])


SINR, RSRP -> RSRP


/Users/ihneskarbovik/Documents/master/master-greier/master/LSTM.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[features] = scaler.fit_transform(X[features])


RSRP, RSRQ -> RSRP


...

/Users/ihneskarbovik/Documents/master/master-greier/master/LSTM.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[features] = scaler.fit_transform(X[features])


SINR, RSRQ -> SINR


/Users/ihneskarbovik/Documents/master/master-greier/master/LSTM.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[features] = scaler.fit_transform(X[features])


SINR, RSRP, campaign -> SINR


ValueError: could not convert string to float: 'Interactivity_gaming_4G5G_TIM_Driving_Palasport_to_Via_Appia_End_No_Flight_Mode'

In [9]:
df_results = pd.DataFrame.from_dict(results, orient='index')

df_results.reset_index(inplace=True)
df_results.rename(columns={'index': 'Combination'}, inplace=True)
df_results = df_results.drop(['y_pred', 'y_true', 'loss', 'val_loss', 'LSTM', 'scaler_pred'], axis=1)
df_results

,Combination,loss_final,val_loss_final,mad,mad?
0,RSRP -> RSRP,0.00062,0.00074,6.2,0.6
1,"SINR, RSRP -> RSRP",0.00061,0.00071,5.7,0.7
2,"RSRP, RSRQ -> RSRP",0.00058,0.00048,6.4,0.5
3,"SINR, RSRP, RSRQ -> RSRP",0.00061,0.00068,5.8,0.5
4,"RSRP, SSS_RSRP -> RSRP",0.00062,0.00058,6.3,0.6
5,"SINR, RSRP, RSRQ, SSS_RSRP, SSS_SINR, SSS_RSRQ...",0.00058,0.00057,5.6,0.6
6,SINR -> SINR,0.00108,0.00122,6.0,0.9
7,"SINR, RSRQ -> SINR",0.00107,0.00100,6.3,1.2
8,"SINR, RSRP -> SINR",0.00108,0.00123,6.3,0.7
9,"SINR, RSRP, RSRQ -> SINR",0.00107,0.00115,6.3,0.7


## Classification of mode using Random Forest

In [ ]:
X, mode = read_dataset('datasets/Op1_merged.csv', campaign=campaign, features=['RSRP', 'RSRQ'])
feature_selection = [['SINR', 'RSRQ'], ['SINR', 'RSRP'], ['RSRP', 'RSRQ'], ['RSRP', 'SINR', 'RSRQ'],
                     ['RSRP', 'SSS_RSRP'], ['SINR', 'SSS_SINR'], ['RSRQ', 'SSS_RSRQ'],
                     ['RSRP', 'SINR', 'RSRQ', 'SSS_RSRP', 'SSS_SINR', 'SSS_RSRQ']]

rf_results = {}

for features in feature_selection:
    used_features = ', '.join(features)
    feats = features.copy()
    X, mode = read_dataset('datasets/Op1_merged.csv', campaign=campaign, features=features)
    rf_results[used_features] = rf(X, mode)

rf_results_df = pd.DataFrame.from_dict(rf_results, orient='index')
rf_results_df.reset_index(inplace=True)
rf_results_df.rename(columns={'index': 'Combination'}, inplace=True)
rf_results_df = rf_results_df.drop(['y_true', 'y_pred', 'model'], axis=1)
rf_results_df

,Combination,Accuracy,Precision,Recall
0,"SINR, RSRQ",0.915194,0.950704,0.733696
1,"SINR, RSRP",0.913175,0.917391,0.758993
2,"RSRP, RSRQ",0.913680,0.943182,0.739750
3,"RSRP, SINR, RSRQ",0.917718,0.948198,0.750446
4,"RSRP, SSS_RSRP",0.974255,0.983607,0.917782
5,"SINR, SSS_SINR",0.964664,0.956604,0.915162
6,"RSRQ, SSS_RSRQ",0.961131,0.955769,0.901996
7,"RSRP, SINR, RSRQ, SSS_RSRP, SSS_SINR, SSS_RSRQ",0.978799,0.967557,0.953008


In [11]:
X, mode = read_dataset('datasets/Op1_merged.csv', campaign=campaign, features=['RSRP', 'RSRQ'])
feature_selection = [['SINR', 'RSRQ'], ['SINR', 'RSRP'], ['RSRP', 'RSRQ'], ['RSRP', 'SINR', 'RSRQ']]

rf_results = {}

for features in feature_selection:
    used_features = ', '.join(features)
    feats = features.copy()
    X, mode = read_dataset('datasets/Op1_merged.csv', campaign=campaign, features=features)
    metrics = rf(X, mode)
    # metrics['pred_y_pred'] = metrics['model'].predict()

rf_results_df = pd.DataFrame.from_dict(rf_results, orient='index')
rf_results_df.reset_index(inplace=True)
rf_results_df.rename(columns={'index': 'Combination'}, inplace=True)
rf_results_df = rf_results_df.drop(['y_true', 'y_pred', 'model'], axis=1)
rf_results_df

KeyError: "['y_true', 'y_pred', 'model'] not found in axis"